## 这个文件用于绘制通过PINN模型对CVAE生成的多模态解进行绘图。

In [3]:
from time import time
from datetime import datetime
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import random, genfromtxt
from IPython.display import display
from matplotlib import rc
from matplotlib.pyplot import figure
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F
import matplotlib.ticker as mticker
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.font_manager as fm
import matplotlib as mpl   

In [4]:
current_directory = os.getcwd()
print(current_directory)
device=torch.device("cuda")
# 检查是否有可用的 GPU
if torch.cuda.is_available():
    device = torch.device("cuda")  # 使用 GPU
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")  # 使用 CPU
    print("No GPU available, using CPU instead.")

/share/users/fxy/work/Constitutive_Equation/MFNN/singlefluid
Using GPU: NVIDIA GeForce RTX 3090


In [5]:

%config InlineBackend.figure_format = 'svg'
# 检查系统是否安装了 Arial 字体
def is_arial_available():
    for font in fm.fontManager.ttflist:
        if 'Arial' in font.name:
            return True
    return False

# 如果系统没有 Arial 字体，加载用户自定义字体
if not is_arial_available():
    user_font_path = os.path.expanduser('~/.local/share/fonts/ARIAL.TTF')
    if os.path.exists(user_font_path):
        # 添加用户字体到 Matplotlib 的字体管理器
        fm.fontManager.addfont(user_font_path)
        # 设置 Matplotlib 使用该字体
        plt.rcParams['font.family'] = 'sans-serif'
        plt.rcParams['font.sans-serif'] = ['Arial', 'DejaVu Sans', 'Liberation Sans']
        print("已加载用户自定义 Arial 字体")
    else:
        print("未找到用户自定义 Arial 字体文件")
else:
    print("系统已安装 Arial 字体")

# 重置 Matplotlib 的全局设置
mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams['font.family'] = ['Arial']  # 设置字体为 Arial
plt.rcParams['font.size'] = 18  # 设置全局字体大小
    


已加载用户自定义 Arial 字体


In [10]:
# 计算哈达玛积
def add_hadamard_features(X):
    """
    计算哈达玛积并拼接新特征。

    参数:
        X (numpy.ndarray): 输入的特征矩阵，形状为 (n_samples, n_features)。
                          列顺序必须为 ['AngFreq', 'Mn1', 'Mn2', 'Mn3', 'Mn11', 'Mn22', 'Mn33']。

    返回:
        numpy.ndarray: 处理后的特征矩阵，形状为 (n_samples, 4)。
                      列顺序为 ['AngFreq', 'Mn1*Mn11', 'Mn2*Mn22', 'Mn3*Mn33']。
    """
   # 计算哈达玛积
    Mn1_Mn11 = X[:, 1] * X[:, 4]  # Mn1 * Mn11
    Mn2_Mn22 = X[:, 2] * X[:, 5]  # Mn2 * Mn22
    Mn3_Mn33 = X[:, 3] * X[:, 6]  # Mn3 * Mn33

    # 计算交叉相乘
    feature_1_2 = Mn1_Mn11 * Mn2_Mn22  # (Mn1 * Mn11) * (Mn2 * Mn22)
    feature_1_3 = Mn1_Mn11 * Mn3_Mn33  # (Mn1 * Mn11) * (Mn3 * Mn33)
    feature_2_3 = Mn2_Mn22 * Mn3_Mn33  # (Mn2 * Mn22) * (Mn3 * Mn33)
    feature_1_2_3=Mn1_Mn11*Mn2_Mn22*Mn3_Mn33
    # 拼接新特征
    X_new = np.column_stack((
        X[:, 0],  # AngFreq
        Mn1_Mn11,
        Mn2_Mn22,
        Mn3_Mn33,
        feature_1_2,
        feature_1_3,
        feature_2_3
    ))
    return X_new

In [11]:
#获取原始数据（主要是获取原本训练时的归一化参数，理论上不应该这样处理，另外一个是获取b4对应的频率-损耗角序列）

# 设置数据类型
DTYPE = torch.float32

# 设置随机种子
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
random.seed(SEED)

df = {}
# 读取 Excel 文件中的所有 sheet
url_hf = 'Data/DifferComponent_data.xlsm'
df_HF = pd.read_excel(url_hf, sheet_name=None)
# 初始化训练集和验证集
train_set = pd.DataFrame()  # 用于存储训练集
valid_set = pd.DataFrame()  # 用于存储验证集

# 遍历所有 sheet
for sheet_name, df in df_HF.items():
    # 移除包含 NaN 的行
    df = df.dropna()
    
    # 根据 sheet_name 划分数据集
    if sheet_name == 'b4': 
        valid_set = df  
    if sheet_name!='b1':
        train_set = pd.concat([train_set, df], ignore_index=True)  # 其他 sheet 拼接为训练集

# 提取目标变量
y1_train = train_set['lossF'].to_numpy()
y1_valid = valid_set['lossF'].to_numpy()

# 提取特征变量
feature_columns = ['AngFreq', 'Mn1', 'Mn2', 'Mn3',  
                   'Mn11', 'Mn22', 'Mn33']
X_train = train_set[feature_columns].to_numpy()
X_valid = valid_set[feature_columns].to_numpy()


X_train=add_hadamard_features(X_train)
X_valid=add_hadamard_features(X_valid)
# 对频率特征进行对数化处理
X_train[:, 0] = np.log10(X_train[:, 0])  # AngFreq 是第一个特征
X_valid[:, 0] = np.log10(X_valid[:, 0])
# 初始化 MinMaxScaler
x_scaler = MinMaxScaler()  # 用于特征 X 的归一化
y_scaler = MinMaxScaler()  # 用于目标 y 的归一化

# 对 X_train 进行归一化
X_train_normalized = x_scaler.fit_transform(X_train)

# 对 y_train 进行归一化
# 注意：y_train 需要 reshape 为二维数组，因为 MinMaxScaler 接受二维输入
y_train_normalized = y_scaler.fit_transform(y1_train.reshape(-1, 1))

# 使用保存的归一化参数对 X_valid 进行归一化
X_valid_normalized = x_scaler.transform(X_valid)
# 使用保存的归一化参数对 y_valid 进行归一化
y_valid_normalized = y_scaler.transform(y1_valid.reshape(-1, 1))

# 将归一化后的数据转换为 PyTorch 张量
X_data_HF = torch.tensor(X_train_normalized, dtype=torch.float32)
y_data_HF = torch.tensor(y_train_normalized, dtype=torch.float32)
X_data_valid = torch.tensor(X_valid_normalized, dtype=torch.float32)
y_data_valid = torch.tensor(y_valid_normalized, dtype=torch.float32)

# 打印张量的形状以确认
print("X_data_HF shape:", X_data_HF.shape)
print("y_data_HF shape:", y_data_HF.shape)
print("X_data_valid shape:", X_data_valid.shape)
print("y_data_valid shape:", y_data_valid.shape)

# 定义模型
in_dim, out_dim = 7, 1

X_data_HF shape: torch.Size([640, 7])
y_data_HF shape: torch.Size([640, 1])
X_data_valid shape: torch.Size([36, 7])
y_data_valid shape: torch.Size([36, 1])


In [14]:
# 读取生成的标签数据并转换为numpy数组
generated_labels = pd.read_excel('Data/generated_labels.xlsx', sheet_name='Generated Labels')
label_columns = ['Mn1', 'Mn2', 'Mn3', 'Mn11', 'Mn22', 'Mn33']
generated_labels_array = generated_labels[label_columns].to_numpy()
print("成功加载生成的标签数据,数据形状为:", generated_labels_array.shape)
print("前5行数据为:\n", generated_labels_array[:5])

成功加载生成的标签数据,数据形状为: (100, 6)
前5行数据为:
 [[19.24592718 52.19185764 79.01475178 10.93694987 20.16281711 28.81637722]
 [20.90636851 52.32290053 79.24331445  9.34296138 20.44041671 30.77089764]
 [19.90967153 52.34865365 78.65866532 10.72613774 19.16339813 28.16293688]
 [20.41629038 53.5296622  79.73968513  9.81620437 19.83479733 30.57848702]
 [19.55662918 52.95986368 79.12772338 10.3691067  19.775179   29.90009443]]
